In [ ]:
# TODO: Load file
#       Find and isolate code for rigid tapping
#       Interpret information from isolated code
#       Print tapping pitch in imperial and metric


In [2]:
import os
from rich import print

# Anatomy of gcode:
# Optional stop
# Tool change
# Spindle speed / rotation
# Positioning offsets/ coolant
# Positioning
# Canned cycle

# Should probably:
# Find each instance of G84 tapping cycle,
# Collect feed value,
# Scroll up until, record closest preceeding rpm and if M29 has been used
# Stop scrolling at tool change M06


In [49]:
# Open file in read mode
with open("1001.nc", 'r') as file:
    lines = file.readlines()

# Find all lines with tool changes
toolChangeIndexes = []
for lineIndex, line in enumerate(lines):
    if line.find("M06") != -1:
        toolChangeIndexes.append(lineIndex)
        # print(lineIndex, line)
# print("tool change indexes:", toolChangeIndexes)
# This above may be completly un needed lmao


# Determine if each tool change has a tapping cycle or not
rigidTappingIndexes = []
for lineIndex, line in enumerate(lines):
    if line.find("G84") != -1:
        rigidTappingIndexes.append(lineIndex+1)
        # print(lineIndex, line)
print("rigid tapping indexes:", rigidTappingIndexes)

# Now that we know where the rigid tapping is called we can go to each index and scroll up until we read a M06 line, and we will interpret the data from each line as we go
allData = []

for tappingCycle in rigidTappingIndexes:
    feedPerRev = False
    Feed = 0
    FeedFound = False
    RPM = 0
    RPMFound = False
    Tool = 0
    toolFound = 0
    
    readLine = tappingCycle

    while lines[readLine].find("M06") == -1:
        # print(lines[readLine])
        readLine = readLine - 1
        if lines[readLine].find("G95") != -1:
            feedPerRev = True
        if lines[readLine].find("F") != -1 and FeedFound == False:
            Feed = lines[readLine][lines[readLine].index('F')+1:].strip()
            Feed = float(Feed)
            FeedFound = True
        if lines[readLine].find("S") != -1 and RPMFound == False:
            RPM = lines[readLine][lines[readLine].index('S')+1:].strip()
            RPM = float(RPM)
            RPMFound = True
        if lines[readLine].find("T") != -1 and toolFound == False:
            Tool = lines[readLine][lines[readLine].index('T')+1:].strip()
            Tool = Tool.replace("M06",'')
            Tool = int(Tool)
            RPMFound = True

    if feedPerRev == True:
        iPitch = Feed
        mPitch = Feed*25.4
    else:
        iPitch = RPM/Feed
        mPitch = Feed/RPM*25.4

    data = {"ToolNum": str(Tool),
            "RPM":str(RPM),
            "Feed":str(Feed),
            "Feed/Rev":feedPerRev,
            "Metric Pitch": str(mPitch),
            "Imperial Pitch": str(iPitch)}
    allData.append(data)
    # print(data)    
    # print("read lines:",tappingCycle,"to",readLine)
    # print(lines[readLine:tappingCycle]) 

allData.reverse()
# print(allData)
    


rigid tapping indexes:
[110, 227, 282, 353, 390, 427, 504, 588, 688, 781]

In [50]:
from rich.console import Console
from rich.table import Table

console = Console()
table = Table(show_header=True)
table.add_column("Tool")
table.add_column("RPM")
table.add_column("Feed")
table.add_column("Metric Pitch")
table.add_column("Imperial Pitch")
for tool in allData:
    table.add_row(tool["ToolNum"],tool["RPM"],tool["Feed"],tool["Metric Pitch"],tool["Imperial Pitch"])
console.print(table)

┏━━━━━━┳━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ Tool ┃ RPM   ┃ Feed ┃ Metric Pitch        ┃ Imperial Pitch     ┃
┡━━━━━━╇━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│ 3    │ 500.0 │ 7.9  │ 0.40132             │ 63.291139240506325 │
│ 5    │ 500.0 │ 9.8  │ 0.49784000000000006 │ 51.0204081632653   │
│ 8    │ 500.0 │ 13.8 │ 0.70104             │ 36.231884057971016 │
│ 11   │ 500.0 │ 15.7 │ 0.7975599999999999  │ 31.847133757961785 │
│ 14   │ 500.0 │ 19.7 │ 1.0007599999999999  │ 25.38071065989848  │
│ 27   │ 500.0 │ 25.0 │ 1.27                │ 20.0               │
│ 29   │ 500.0 │ 15.6 │ 0.79248             │ 32.05128205128205  │
│ 17   │ 500.0 │ 19.7 │ 1.0007599999999999  │ 25.38071065989848  │
│ 20   │ 500.0 │ 24.6 │ 1.24968             │ 20.32520325203252  │
│ 25   │ 500.0 │ 29.5 │ 1.4986              │ 16.949152542372882 │
└──────┴───────┴──────┴─────────────────────┴────────────────────┘